# So simple a child could do it

My colleague came to me with this problem, taken from their 9-year-old boy who we will call Johnny.

Johnny was found to be despairing over this particular homework question.

> With the numbers 1,2,3,4,5 make a 2 digit and 3 digit number (only using each number once) that gives:  
> a) the smallest product  
> b) the largest product
> 
> For example:
>
> $12 \times 345 = 4140$  
> $21 \times 435 = 9135$  
> $34 \times 251 = 8534$

I sympathise; I think if someone used the word "product" around me when I was 9 I'd run for the hills.

Much like the maligned [New Math](https://www.youtube.com/watch?v=9mc7eb1i9o4) of the 60s this is a style of mathematical education that attempts to engage children in lateral thinking. [The many varieties of this problem](https://www.youtube.com/watch?v=BpzkefmyQLMB) focus on teaching how to hone in on an answer through soft deduction.

This is by far my least favourite kind of exercise: a whole lot of guesswork for not much payoff. I find it quite likely that Johnny feels the same way, so it would be nice if there were some rules that could be applied instead of taking stabs in the dark.

Well, there _are_. These take the guesswork out entirely, regardless of the five unique digits chosen.
I will demonstrate how, but first let's figure out what the answer is to this question.

## Naive solution

There are 120 total permutations in this question. Poor Johnny better sharpen his pencils!

The reason we can say there are 120 permutations is because at the core we're asking in how many ways can `[1, 2, 3, 4, 5]` be rearranged
without duplication, which is $5!$ (5 factorial).

To illustrate this point, just image we have a list of permutations of `[1, 2, 3, 4, 5]` like so:

$$1 \, 2 \, 3 \, 4 \, 5$$

$$2 \, 1 \, 4 \, 3 \, 5$$

$$3 \, 4 \, 2 \, 5 \, 1$$

$$\dots$$

If we introduce $\times$ between the second and third digits we now have the multiplication expression:

$$12 \times 345$$

$$21 \times 435$$

$$34 \times 251$$

$$\dots$$

So if there are 120 permutations of `[1, 2, 3, 4, 5]` then there must be 120 permutations of the multiplication expression.

A terrible toll on Johnny, but computers don't complain. So let's use brute-force to find the largest/smallest permutations.

In [95]:
# Brute-force solution to finding the largest/smallest products 
# in the `aa * bbb` problem.

from itertools import permutations
from functools import reduce

def to_product_triple_2_3(a_t, a_u, b_h, b_t, b_u):
    """
    Given: a_t, a_u, b_h, b_t, b_u, we combine the a and b 
    parts into a 2-digit and 3-digit number respectively,
    returning the resultant numbers and their product as a 
    tuple `(aa, bbb, aa * bbb)`.
    """
    aa = 10 * a_t + a_u
    bbb = 100 * b_h + 10 * b_t + b_u
    return (aa, bbb, aa * bbb)

# From a list of tuples of form (aa, bbb, aa * bbb), find the tuple with the smallest product.
find_smallest_product = lambda product_list: reduce(lambda x, y: x if x[2] < y[2] else y, product_list)

# From a list of tuples of form (aa, bbb, aa * bbb), find the tuple with the largest product.
find_largest_product = lambda product_list: reduce(lambda x, y: x if x[2] > y[2] else y, product_list)

# Generate the list of all possible products.
perms = permutations([1, 2, 3, 4, 5])
products = [to_product_triple_2_3(*x) for x in perms]
print("Total number of permutations:", len(products))

# Prettify the tuple
fmt_perm = lambda x: "{0} x {1} = {2}".format(*x)

def print_smallest_product(small_list):
    print("Smallest product is:", fmt_perm(find_smallest_product(small_list)))
print_smallest_product(products)

def print_largest_product(large_list):
    print("Largest product is: ", fmt_perm(find_largest_product(large_list)))
print_largest_product(products)

Total number of permutations: 120
Smallest product is: 13 x 245 = 3185
Largest product is:  52 x 431 = 22412


My colleague wrote a python script similar to this to help solve Johnny's homework (we just have to hope the teacher wasn't expecting him to show his working.)

## Beyond brute-force

Of course this isn't how Johnny is expected to solve the problem. Johnny is expected to reasonably order the numbers to produce small and large products. How might he arrange the numbers?

Well let's compare two permutations:

> $12 \times 345 = 4140$  
> $21 \times 345 = 7245$

Notice that the same digits occupy the two-digit number and three-digit number respectively, the only difference is the _local_ ordering of the two-digit number.

It's pretty obvious that $12 < 21$. So it also stands to reason that that $(12 \times y) < (21 \times y)$ for some constant $y$.

We can extrapolate this to the three digit number also: $345 < 354 < 435 < 453 < 534 < 543$. Given `[3, 4, 5]` the smallest product will be a multiplication of 345, and the largest product will be a multiplication of 543.

So given a number made of the digits `[1, 2]`, and another number made of the digits `[3, 4, 5]` then the smallest possible product must be $12 \times 345$, and the largest product from must be $21 \times 543$.

There is a general rule here:

* For the **smallest** product, the digits in a number must be sorted in **ascending** order, starting from the most significant column.
* For the **largest** product, the digits in a number must be sorted in **descending** order, starting from the most significant column.

### Calculating combinations

How does this help us? It lets us discard a significant number of permutations; the order in which the digits are arranged must conform to the above rules.

To demonstrate this, consider the following:

> Imagine you have 5 ping-pong balls labelled with the numbers 1-5 placed in a bag. 
> If you then draw two balls at random from the bag these will form your 2-digit number.
> The remaining 3 balls in the bag will form your 3-digit number. 
> 
> How many combinations are there?

Notice that the order in which we draw from the bag does not matter here. Drawing `1` then `2` is exactly the same as drawing `2` then `1`. Here is our solution:

```python
[1, 2], [3, 4, 5]
[1, 3], [2, 4, 5]
[1, 4], [2, 3, 5]
[1, 5], [2, 3, 4]
[2, 3], [1, 4, 5]
[2, 4], [1, 3, 5]
[2, 5], [1, 3, 4]
[3, 4], [1, 2, 5]
[3, 5], [1, 2, 4]
[4, 5], [1, 2, 3]
```

10 possible combinations.

Bear in mind we have already seen that there is only one way that each combination can be arranged to give the smallest possible product (sort the digits in ascending order); and there is only one way in which each product can produce largest possible solution (sort the digits in descending order). This means that in total there are only 20 permutations that we need to focus on: 10 for the smallest product, and 10 for the largest.

Instead of calculating all the products by hand, let's have Python do this bit for us.

In [82]:
from itertools import combinations

digit_list = [1, 2, 3, 4, 5]

def get_smallest_largest_perms_2_3(digit_list):
    """
    From an ordered list of 5 digits, returns the lists of smallest and largest products in a tuple of the form:
    (smallest_list, largest_list).
    """
     # List of possible two digit numbers.
    two_digits_list = list(combinations(digit_list, 2))

    # List of possible 3-digit numbers that are compliments to the 2-digit numbers.
    three_digits_list = [[y for y in digit_list if y not in two_digits] for two_digits in two_digits_list]

    # Calculate the smallest possible products from the combination of the 2-digit and 3-digit number.
    # Note: the number digits are already sorted in ascending order as `combinations` guarantees ordering is preserved.
    smallest_list = [to_product_triple_2_3(*x, *y) for (x, y) in zip(two_digits_list, three_digits_list)]

    # Calculate the largest possible products from the combination of the 2-digit and 3-digit number.
    # Note: we reverse the digit order of the 2-digits and 3-digit numbers to achieve this.
    largest_list = [to_product_triple_2_3(*x[::-1], *y[::-1]) for (x, y) in zip(two_digits_list, three_digits_list)]
    
    return smallest_list, largest_list

smallest_list, largest_list = get_smallest_largest_perms_2_3(digit_list)

# Show our calculations.
print("{0:20} {1}".format("Smallest products", "Largest products"))
for (s, l) in zip(smallest_list, largest_list):
    print("{0:20} {1}".format(fmt_perm(s), fmt_perm(l)))
print("=====================================")
print_smallest_product(smallest_list)
print_largest_product(largest_list)

Smallest products    Largest products
12 x 345 = 4140      21 x 543 = 11403
13 x 245 = 3185      31 x 542 = 16802
14 x 235 = 3290      41 x 532 = 21812
15 x 234 = 3510      51 x 432 = 22032
23 x 145 = 3335      32 x 541 = 17312
24 x 135 = 3240      42 x 531 = 22302
25 x 134 = 3350      52 x 431 = 22412
34 x 125 = 4250      43 x 521 = 22403
35 x 124 = 4340      53 x 421 = 22313
45 x 123 = 5535      54 x 321 = 17334
Smallest product is: 13 x 245 = 3185
Largest product is:  52 x 431 = 22412


This is much more manageable, but we promised Johnny that we would find a solution that remove the laborious guesswork. So let's take it a step further.

## Beyond `[1, 2, 3, 4, 5]`

We know the answer for digits `[1, 2, 3, 4, 5]`: 
$13 \times 245$ and $52 \times 431$ for the smallest and largest products respectively.

Let's try out some other digit combos and see what we get. Maybe there's a pattern in `[2, 3, 6, 8, 9]` and `[1, 6, 7, 8, 9]`. 



In [83]:
list_23689 = [2, 3, 6, 8, 9]
smallest_23689, largest_23689 = get_smallest_largest_perms_2_3(list_23689)
print("For list:", list_23689)
print_smallest_product(smallest_23689)
print_largest_product(largest_23689)

print("")

list_16789 = [1, 6, 7, 8, 9]
smallest_16789, largest_16789 = get_smallest_largest_perms_2_3(list_16789)
print("For list:", list_16789)
print_smallest_product(smallest_16789)
print_largest_product(largest_16789)


For list: [2, 3, 6, 8, 9]
Smallest product is: 26 x 389 = 10114
Largest product is:  93 x 862 = 80166

For list: [1, 6, 7, 8, 9]
Smallest product is: 17 x 689 = 11713
Largest product is:  96 x 871 = 83616


There is a pattern here. Can you see it?
It seems to be the case that the same indices are always used to construct the smallest and largest products! 

Look carefully at `[2, 3, 6, 8, 9]` the smallest product's 2-digit number is 26; the 2-digit number is formed from the first and third digits from the list. For `[1, 6, 7, 8, 9]` the smallest product's 2-digit number is 17, the 2-digit number is again formed by the first and third digits in the list!

The exact same pattern is seen for the largest product, only this time it is the fifth and and second digits.


We can write this more formally as:

Given the digits:
$d_1, d_2, d_3, d_4, d_5 \in \left\{1, 2, 3, 4, 5, 6, 7, 8, 9\right\}$ where $d_1 < d_2 < d_3 < d_4 < d_5$,  
it is the case that for any combination of the digits into a 2-digit and 3-digit number, the smallest product $s$ is:

$$
d_1d_3 = d_2d_4d_5 = s
$$

and likewise, the the largest product $l$ is:

$$
d_5d_2 = d_4d_3d_1 = l
$$

This is another computationaly quick task. So we can go through all possible combinations of digits to prove this theory:

In [100]:
def calculated_smallest_product_2_3(li):
    """
    Given an ordered list of 5 digits, return the smallest product of the calculation aa * bbb. 
    Return value is a tuple of (aa, bbb, aa * bbb)
    """
    return to_product_triple_2_3(li[0], li[2], li[1], li[3], li[4])

def calculated_largest_product_2_3(li):
    """
    Given an ordered list of 5 digits, returns the largest product of the calculation aa * bbb.
    Return value is a tuple of (aa, bbb, aa * bbb)
    """
    return to_product_triple_2_3(li[4], li[1], li[3], li[2], li[0])

def test_permutations_combo(digit_list):
    """
    Given an ordered list of 5 digits (i.e. `[1, 2, 3, 4, 5]`), iterates through all possible 
    permutations of `aa * bbb` and verifies that `calculated_smallest_product` and 
    `calculated_largest_product` produce the correct output.
    """
    smallest_list, largest_list = get_smallest_largest_perms_2_3(digit_list)
    
    smallest_match = calculated_smallest_product_2_3(digit_list) == find_smallest_product(smallest_list)    
    largest_match = calculated_largest_product_2_3(digit_list) == find_largest_product(largest_list)
    
    return smallest_match and largest_match

# Go through all possible combinations of taking 5 digits from the range 1-9, and calculate that 
# for all given digits that `calculated_smallest_product_2_3` and `calculated_largest_product_2_3` holds.
all_digit_lists =  list(combinations([1, 2, 3, 4, 5, 6, 7, 8, 9], 5))
print("Number of lists of digits to test:", len(all_digit_lists))
print("Theorem holds for all possible inputs:", all(map(test_permutations_combo, all_digit_lists)))


Number of lists of digits to test: 126
Theorem holds for all possible inputs: True


Fantastic news!

So in the end we don't need to do a linear search of this search-space, regardless of the combinations. This can be solved in $O(1)$.

# Conclusion

Little Johnny can rejoice, given an ordered list of 5 unique digits between 1 and 9, (e.g., `[1, 2, 3, 4, 5]`, `[2, 3, 7, 8, 9]`, etc.) it is **always** the case that the smallest product can be calculated by: $d_1d_3 = d_2d_4d_5$, and the latest product by $d_5d_2 = d_4d_3d_1$.

### Examples

* The smallest product of a 2-digit by 3-digit number in `[1, 2, 7, 8, 9]` _must_ be $17 \times 289$.
* The largest product of a 2-digit by 3-digit number in `[3, 4, 5, 6, 7]` _must_ be $74 \times 653$.
* And 250 more varieties!

So never fear, Johnny, because if you are faced with such a question again then just remember this ordering rule.